In [5]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

def getPageHtml(url):
    # Chrome WebDriver 옵션 설정
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # 브라우저 창이 표시되지 않도록 설정
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    #options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36")
    
    # Chrome WebDriver 실행
    driver = webdriver.Chrome('chromedriver', options=options)

    # 페이지 열기
    driver.get(url)

    # 페이지의 초기 높이 가져오기
    scroll_pause_height = driver.execute_script("return document.documentElement.scrollHeight")
    
    # 페이지 충분히 스크롤
    scroll_pause_time = 0.5  # 스크롤링 사이의 일시 정지 시간 (초)
    while True:
        # 스크롤을 아래로 이동
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

        # 일시 정지
        time.sleep(scroll_pause_time)

        # 새로운 스크롤 높이 가져오기
        new_scroll_height = driver.execute_script("return document.documentElement.scrollHeight")

        # 더 이상 스크롤링이 되지 않으면 종료
        if new_scroll_height == scroll_pause_height:
            break

        # 스크롤 높이 업데이트
        scroll_pause_height = new_scroll_height

    # 페이지의 HTML 가져오기
    html = driver.page_source

    # BeautifulSoup을 사용하여 HTML 파싱
    soup = BeautifulSoup(html, "html.parser")

    # WebDriver 종료
    driver.quit()

    return soup



def getUrl(dfrow):
    
    row = dfrow.copy()
    # Chrome WebDriver 옵션 설정
    options = webdriver.ChromeOptions()
    if row['mall'] != '쿠팡':
        options.add_argument("--headless")  # 브라우저 창이 표시되지 않도록 설정
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")    
    
    url = row['링크']

    driver = webdriver.Chrome('chromedriver', options=options)
    driver.get(url)
    #리디렉션 완료될 때까지 기다리기
    while 'https://cr.shopping.naver.com/'in driver.current_url:
        time.sleep(0)
    time.sleep(.1)
    html = driver.page_source
    source = BeautifulSoup(html, "html.parser")
    nlink = driver.current_url

    driver.quit()

    return (source, nlink)

In [9]:


def get_mall_name(df):
    selector = {'쿠팡':'div[class="prod-sale-vendor"] a', 'G마켓':'span[class*="seller"]', '11번가':'h1[class*="store_title"]', '옥션':'span[class*="text__seller"]'}
    length = str(len(df))
    print(length+'개의 목록 작업 시작')
    rlist = []
    for i in range(len(df)):
        print(str(i+1)+' 번째')
        row = df.iloc[i].copy()
        if row['mall'] in ['쿠팡', 'G마켓', '11번가', '옥션']:
            print(row['mall'])
            (soup, nlink) = getUrl(row)
            shopInfo = soup.select_one(selector[row['mall']])
            if shopInfo:
                shop = shopInfo.text
            else:
                shop = ''
            row['링크'] = nlink
            row['platform'] = row['mall']
            row['mall'] = shop
        rlist.append(row)
    ndf = pd.DataFrame(rlist)

    return ndf

gdf = get_mall_name(gdf)
gdf

7개의 목록 작업 시작
1 번째
G마켓


WebDriverException: Message: unknown error: Chrome failed to start: was killed.
  (unknown error: DevToolsActivePort file doesn't exist)
  (The process started from chrome location C:\Program Files\Google\Chrome\Application\chrome.exe is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x01168893+48451]
	(No symbol) [0x010FB8A1]
	(No symbol) [0x01005058]
	(No symbol) [0x01023211]
	(No symbol) [0x01020019]
	(No symbol) [0x01050798]
	(No symbol) [0x0105047C]
	(No symbol) [0x0104A0B6]
	(No symbol) [0x01027E08]
	(No symbol) [0x01028F2D]
	GetHandleVerifier [0x013C8E3A+2540266]
	GetHandleVerifier [0x01408959+2801161]
	GetHandleVerifier [0x0140295C+2776588]
	GetHandleVerifier [0x011F2280+612144]
	(No symbol) [0x01104F6C]
	(No symbol) [0x011011D8]
	(No symbol) [0x011012BB]
	(No symbol) [0x010F4857]
	BaseThreadInitThunk [0x763700C9+25]
	RtlGetAppContainerNamedObjectPath [0x77757B4E+286]
	RtlGetAppContainerNamedObjectPath [0x7775

In [4]:
urlList = ["https://search.shopping.naver.com/search/all?frm=NVSHATC&origQuery=D10XPA&pagingIndex=1&pagingSize=80&productSet=total&query=D10XPA&sort=rel&timestamp=&viewType=list",
           "https://search.shopping.naver.com/search/all?frm=NVSHATC&origQuery=D10XPA&pagingIndex=2&pagingSize=80&productSet=total&query=D10XPA&sort=rel&timestamp=&viewType=list"]
alist = []
for url in urlList:
    soup = getPageHtml(url)
    tag = soup.select_one('#content > div.style_content__xWg5l > div.basicList_list_basis__uNBZx')
    alist += tag.select('div[class*="product_item__"]')


In [6]:
def get_prod_list_df(alist):
    import pandas as pd
    import re
    rowdata = []
    for i in alist:
        if i.select_one('span[class*="price_delivery"]'):
            title = i.select_one('div[class*="title"] > a[href]')
            link = title['href']
            title = title.text
            mall = i.select_one('div[class*="mall_title"] > a[class*="mall"]')
            mallname = mall.text
            if not mallname:
                mallname = mall.select_one('img[alt]')['alt']
            
            price = i.select_one('span[class*="price_price"]').text    
            price = int(re.sub(r'[^0-9]', '', price))
            delivery = i.select_one('span[class*="price_delivery"]').text    
            delivery = re.sub(r'[^0-9]', '', delivery)
            if delivery=='':
                delivery = 0;
            else:
                delivery = int(delivery)

            sum = int(price)+int(delivery)
            row = {'mall':mallname, '제목':title, '가격':price, '배송비':delivery, '합계':sum, '링크':link}
            rowdata.append(row)
    df = pd.DataFrame(rowdata)
    df['수량'] = df['제목'].str.extract(r'([0-9]+)개')
    df = df.fillna(1)
    df['수량']=df['수량'].astype(int)
    df['단가'] = df['가격']/df['수량']
    df['위반'] = df['단가']<31000
    df = df[df['위반']]
    return df
df = get_prod_list_df(alist)
df

,mall,제목,가격,배송비,합계,링크,수량,단가,위반
0,쿠팡,팬직 TFV-D10XPA 환풍기 욕실용환풍기 화장실용,29000,3000,32000,https://cr.shopping.naver.com/adcr.nhn?x=0mTn1...,1,29000.000000,True
2,쿠팡,팬직 욕실형 환풍기 TFV-D10XPA 구모델 TFV-13BV,29000,3000,32000,https://cr.shopping.naver.com/adcr.nhn?x=LSch9...,1,29000.000000,True
3,G마켓,팬직 욕실 환풍기 팬직 TFV-D10XPA 화장실 천장용 환기팬,27310,2500,29810,https://cr.shopping.naver.com/adcr.nhn?x=1MIRX...,1,27310.000000,True
10,대성풍력,[팬직] 천정용 플라스틱 환풍기 욕실용환풍기 사무실환풍기 TFV-D10XPA,26900,3500,30400,https://cr.shopping.naver.com/adcr.nhn?x=JLcDD...,1,26900.000000,True
15,G마켓,팬직 하늘풍력/팬직/천장용팬/실링팬/환풍기/TFV-D10XPA,25560,5000,30560,https://cr.shopping.naver.com/adcr.nhn?x=d2LcK...,1,25560.000000,True
24,11번가,"팬직 국산욕실환풍기/주방,욕실,화장실용/TFV-D10XPA",28160,0,28160,https://cr.shopping.naver.com/adcr.nhn?x=8lLZ4...,1,28160.000000,True
26,옥션,팬직 TFV-D10XPA 욕실환풍기 화장실환풍기 환기팬,29000,3500,32500,https://cr.shopping.naver.com/adcr.nhn?x=5LB2E...,1,29000.000000,True
33,옥션,팬직/천장용팬/실링팬/환풍기/TFV-D10XPA/욕실용,28900,3000,31900,https://cr.shopping.naver.com/adcr.nhn?x=UwJ6D...,1,28900.000000,True
34,G마켓,팬직/천장용팬/실링팬/환풍기/TFV-D10XPA/욕실용,28900,3000,31900,https://cr.shopping.naver.com/adcr.nhn?x=vWE8%...,1,28900.000000,True
36,11번가,팬직 플라스틱팬 천장용 TFV-D10XPA/환풍기/천정용/송풍기/천지환기,29760,4000,33760,https://cr.shopping.naver.com/adcr.nhn?x=qjWN3...,1,29760.000000,True


In [8]:
cdf = df[df['mall']=='쿠팡']
gdf = df[df['mall']=='G마켓']
edf = df[df['mall']=='11번가']
adf = df[df['mall']=='옥션']
